## Identify Points for StreamStats

__Description__: Tool to identify confluence pair points for tributaries of a specific length, add points to the main stem of a stream network at a specific distance interval, and export a shapefile of the points.

__Input__: Stream grid from the [SteamStats Repository](https://streamstatsags.cr.usgs.gov/StreamGrids/directoryBrowsing.asp), masked using `ClipRaster_withMask.ipynb` and the latitude and longitude of the catchment outlet.

__Output__: A shapefile containing the latitude and longitude of points on the stream grid (confluence and main stem locations).


*Authors*: sputnam@Dewberry.com & slawler@Dewberry.com

### Load libraries and Python options:

In [115]:
import os
import sys
sys.path.append(r'../Core')
from StreamStats_ID_Points import *
import ID_Confluences
import Calc_Distance

### Load the clipped stream grid, specify the catchment outlet, and the tributary exclusion distance:

##### Specify:

In [116]:
# Specify the location of the stream grid raster which was masked by the
# catchment polygon
path = r'P:\02\NY\R2_BLE_Discovery\TECHNICAL\Niagara\HYDROLOGY\streamstats'
# The name of the stream grid raster
name = 'str900_Niagara_buffer.tif'
# The name of the desired output point shapefile
out_shp_name = 'point9'

# latitude of the catchment outlet
lat = 4732395.01795 # [m]
# longitude of the catchment outlet
lon = 227884.624697 # [m]

# Tributary exclusion distance
# disexl = (5280 / 2.0) * (0.3048) # [m] (0.5 miles)
disexl = 30 # [m]

##### Load the stream grid:

In [117]:
# Open the stream grid raster and create an object
sg = StreamGrid(os.path.join(path, name))

# Extract the coordinate reference system value (epsg) for the raster
crs = sg.crs_value()
print("epsg:", crs)

# Create a dataframe from the stream grid data
df = sg.dataframe()

# Potential Issue: Replace 255 with 0, where 255 corresponds to the
# non-stream cells
df.replace(255, 0, inplace=True)
df.head(n=2)

epsg: 26918


,0,1,2,3,4,5,6,7,8,9,...,2521,2522,2523,2524,2525,2526,2527,2528,2529,2530
0,-128,-128,-128,-128,-128,-128,-128,-128,-128,-128,...,-128,-128,-128,-128,-128,-128,-128,-128,-128,-128
1,-128,-128,-128,-128,-128,-128,-128,-128,-128,-128,...,-128,-128,-128,-128,-128,-128,-128,-128,-128,-128


##### Convert the catchment outlet's lat/lon to row/column in the stream grid dataframe and extract the cell size:

In [118]:
# Transform the lat and lon values to the row/column location with the stream
# grid dataframe
pix_x, pix_y = coord2index(sg, lat, lon)
# Add these values to a list as a touple
pourpoint=[(pix_x, pix_y)]
print("Pourpoint XY:", pourpoint)

# Raster cell size
cellsize=sg.cell_size() # [m]
print("The Cell Size:", cellsize)

Pourpoint XY: [(748, 2009)]
The Cell Size: 10.0


### Move up the stream and identify the confluences:

In [119]:
# Function that moves up a stream network from the pourpoint and identifies
# the confluence pairs
save_confluence, confluence_pairs_orig, nogo = ID_Confluences.main(pourpoint,
                                                                   df,
                                                                   display=True)

print("Number of points identified:", len(save_confluence))

Number of points identified: 5


### Remove superflous confluences:

In [120]:
# A false confluence is where there are not two points with the same
# confluence number
true_confluence, false_confluence = Remove_False_Confluence(save_confluence)
print("Number of true confluence points:", len(true_confluence))

Number of true confluence points: 5


##### Identify all stream cells associated with the false confluences:

In [121]:
false_points = ID_False_ConfluenceLocs(false_confluence, nogo)

##### Remove any original confluences associated with the false confluences:

In [122]:
confluence_pairs_orig = Remove_False_From_Orig(false_confluence, confluence_pairs_orig)

### Calculate the tributary length:

In [123]:
tributary, mainstem, nogoabs = Calc_Distance.main(df, cellsize,
                                                  true_confluence,
                                                  false_points,
                                                  confluence_pairs_orig) 

print("Total Confluence Points:", (len(mainstem) + len(tributary)), "Main Stem Points:", len(mainstem))

Total Confluence Points: 5 Main Stem Points: 1


##### Remove confluences with tributaries or intervals less than a specific length:

In [124]:
incl_tribs = Exclude_Confls(tributary, disexl) 

print("# of Tributary Points:", len(incl_tribs)) 

# Note that this is a temporary fix until the distance calculation is fixed so
# that it adds up excluded tribs.
incl_mainstem=Exclude_Confls(mainstem, disexl/2.) 
# Add in the original catchment outlet
incl_mainstem.append((pourpoint[0] + (0, 0,)))

print("# of Main Stem Points:", len(incl_mainstem))   

# of Tributary Points: 4
# of Main Stem Points: 2


##### Add labels to differentiate between the mainstem and the tributaries and combine:

In [125]:
# Empty list to store the labeled main stem points
mainstem_withlabel=[]
# Empty list to store the labeled tributary points 
tributary_withlabel=[]

# Add a "M" to the tuple to indicate that the cell is a main stem location
for cell in incl_mainstem:
    mainstem_withlabel.append(cell+('M',))

# Add a "T" to the tuple to indicate that the cell is a tributary location
for cell in incl_tribs:
    tributary_withlabel.append(cell+('T',))

# Combine the main stem and tributary lists into a singe list
export_confluences=mainstem_withlabel+tributary_withlabel

### Save the results:

##### Extract the confluence number, distance, and type of confluence:

In [126]:
# Empty list to store the confluence number
export_cnum=[]
# Empty list to store the distance
export_dis=[]
# Empty list to store the point type
export_type=[]

# Store the confluence number, distance, and point type.
for cell in export_confluences:
    export_cnum.append(cell[2])
    export_dis.append(cell[3])    
    export_type.append(cell[4])

##### Transform and save as a shapefile:

In [127]:
# Transform the row/column value to latitude/longitude for each confluence
longitude, latitude=index2coord(sg, export_confluences)

# Store the longitude/latitude, confluence number, distance, and type for each
# confluence in a geodataframe
gdf=geodataframe(longitude, latitude, crs, export_cnum, export_dis, export_type)

# Export the geodataframe as a shapefule
gdf.to_file(filename = os.path.join(path, f'{out_shp_name}.shp'))

# End